In [335]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [336]:
import pandas as pd
from functions import main_cleaning

pd.set_option('display.max_columns', None) 

df_raw = pd.read_csv("dielectron.csv")

In [337]:
df_cleaned = main_cleaning(df_raw)
df_cleaned.reset_index(drop=True, inplace=True)

e:\Iván\Documentos\Clase\Informatica\Beca IronHack\Temario\PROYECTOS\Proyecto_final_1710\functions.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_same_charge"] = df["Q1"] == df["Q2"]
e:\Iván\Documentos\Clase\Informatica\Beca IronHack\Temario\PROYECTOS\Proyecto_final_1710\functions.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_outlier"] = False


# MACHINE LEARNING

In [338]:
df_ml = df_cleaned.dropna()

In [339]:
import tensorflow as tf
from tensorflow import keras

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.17.0


In [340]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# import tensorflow as tf
# from tensorflow.keras import layers, models, regularizers
# from sklearn.metrics import root_mean_squared_error, r2_score

# features_lower = df_ml[["Run", "pz1", "Q1", "pz2", "Q2", "is_outlier"]]
# target_lower = df_ml["M"] 

# # División del conjunto de datos en entrenamiento y prueba
# X_train, X_test, y_train, y_test = train_test_split(features_lower, target_lower, test_size=0.20, random_state=42)

# # Normalización de los datos (entre 0 y 1)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Convertir a tensores de TensorFlow
# X_train = X_train_scaled.astype('float32')
# X_test = X_test_scaled.astype('float32')
# y_train = y_train.astype('float32')
# y_test = y_test.astype('float32')

# # Construir el modelo (regresión)
# model = models.Sequential()
# model.add(layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l2(0.01)))
# model.add(layers.Dropout(0.2))  # Añadir dropout para regularización
# model.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
# model.add(layers.Dropout(0.2))  # Añadir dropout para regularización
# model.add(layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
# model.add(layers.Dropout(0.2))  # Añadir dropout para regularización
# model.add(layers.Dense(1))  # Capa de salida para regresión

# # Compilar el modelo
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

# # Entrenar el modelo
# history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test))

# # Evaluar el modelo
# test_loss, test_mae = model.evaluate(X_test, y_test)
# print('Test MAE:', test_mae)

# # Predicciones del conjunto de prueba
# y_pred = model.predict(X_test)

# # Calcular métricas adicionales
# rmse = root_mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f'Test RMSE: {rmse}')
# print(f'Test R²: {r2}')


In [341]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import root_mean_squared_error, r2_score

features_lower = df_ml[["Run", "pz1", "Q1", "pz2", "Q2", "is_outlier"]]
target_lower = df_ml["M"] 

# División del conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(features_lower, target_lower, test_size=0.20, random_state=42)

# Normalización de los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convertir a tensores de TensorFlow
X_train = X_train_scaled.astype('float32')
X_test = X_test_scaled.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

# Construir el modelo (regresión)
model = models.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.BatchNormalization())  # Añadir BatchNormalization
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.BatchNormalization())  # Añadir BatchNormalization
model.add(layers.Dropout(0.2))
model.add(layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.BatchNormalization())  # Añadir BatchNormalization
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1))  # Capa de salida para regresión

# Compilar el modelo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

# Early Stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10000, batch_size=1024, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluar el modelo
test_loss, test_mae = model.evaluate(X_test, y_test)
print('Test MAE:', test_mae)

# Predicciones del conjunto de prueba
y_pred = model.predict(X_test)

# Calcular métricas adicionales
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Test RMSE: {rmse}')
print(f'Test R²: {r2}')


Epoch 1/10000


c:\Anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 1500.2136 - mae: 29.8833 - val_loss: 1511.0404 - val_mae: 30.0835
Epoch 2/10000
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1356.3108 - mae: 29.1164 - val_loss: 1572.0557 - val_mae: 31.9903
Epoch 3/10000
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1167.9021 - mae: 27.5634 - val_loss: 1698.8997 - val_mae: 34.9415
Epoch 4/10000
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 977.6088 - mae: 25.3926 - val_loss: 1366.6003 - val_mae: 31.8113
Epoch 5/10000
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 802.8555 - mae: 22.6409 - val_loss: 986.3462 - val_mae: 26.4807
Epoch 6/10000
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 663.8520 - mae: 19.8194 - val_loss: 657.6002 - val_mae: 20.3779
Epoch 7/10000
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 538.8035 - mae: 16.9836 - val_loss: 454.9933 - val_mae: 15.5702
Epoch 8/10000
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 430.0160 - mae: 14.3874 - val_loss: 349.8752 - val_mae: 12.5496
Epoch 9

In [342]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# import tensorflow as tf
# from tensorflow.keras import layers, models, regularizers
# from sklearn.metrics import root_mean_squared_error, r2_score

# features_lower = df_ml[["Run", "pz1", "Q1", "pz2", "Q2", "is_outlier"]]
# target_lower = df_ml["M"] 

# # División del conjunto de datos en entrenamiento y prueba
# X_train, X_test, y_train, y_test = train_test_split(features_lower, target_lower, test_size=0.20, random_state=42)

# # Normalización de los datos (entre 0 y 1)
# norm = MinMaxScaler()
# X_train_norm = norm.fit_transform(X_train)
# X_test_norm = norm.transform(X_test)

# # Convertir a tensores de TensorFlow
# X_train = X_train_norm.astype('float32')
# X_test = X_test_norm.astype('float32')
# y_train = y_train.astype('float32')
# y_test = y_test.astype('float32')

# # Construir el modelo (regresión)
# model = models.Sequential()
# model.add(layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l2(0.01)))
# model.add(layers.Dropout(0.2))  # Añadir dropout para regularización
# model.add(layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
# model.add(layers.Dropout(0.2))  # Añadir dropout para regularización
# model.add(layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
# model.add(layers.Dropout(0.2))  # Añadir dropout para regularización
# model.add(layers.Dense(1))  # Capa de salida para regresión

# # Compilar el modelo
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

# # Entrenar el modelo
# history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test))

# # Evaluar el modelo
# test_loss, test_mae = model.evaluate(X_test, y_test)
# print('Test MAE:', test_mae)

# # Predicciones del conjunto de prueba
# y_pred = model.predict(X_test)

# # Calcular métricas adicionales
# rmse = root_mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f'Test RMSE: {rmse}')
# print(f'Test R²: {r2}')
